<a href="https://colab.research.google.com/github/dannyyho/ufcstatscraper/blob/master/MMAPredictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/drive')

df_train = pd.read_csv('/drive/My Drive/feedformodel.csv')
df_test = pd.read_csv('/drive/My Drive/week4.csv')
df_test_pred = pd.read_csv('/drive/My Drive/week4.csv')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from google.colab import data_table
data_table.enable_dataframe_formatter()

from google.colab import data_table
from vega_datasets import data

data_table.enable_dataframe_formatter()


# DATA PREPROCESSING

In [3]:
df_train = df_train.replace('--', np.nan)
df_test = df_test.replace('--', np.nan)

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer


# # Simple Imputer for missing fighter stances.
# stance_feats = ['fighter1_stance','fighter2_stance']
# null_feats = ['fighter1_height','fighter1_weight','fighter1_reach','fighter2_height','fighter2_weight','fighter2_reach']

# stance_imputer = SimpleImputer(strategy='most_frequent')

# # stance_imputed_pipeline = stance_imputer.fit(df_train[stance_feats])
# # df_train[stance_feats] = stance_imputed_pipeline.transform(df_train[stance_feats])

In [5]:

# # Iterative Imputation for fighter height, weight, reach, birthyear.

# iter_imputer = IterativeImputer(random_state=42)

# # iter_imputed_pipeline = iter_imputer.fit(df_train[null_feats])
# # df_train[null_feats] = iter_imputed_pipeline.transform(df_train[null_feats])


In [6]:
df_train = df_train.reindex(sorted(df_train.columns), axis=1)
df_test = df_test.reindex(sorted(df_test.columns), axis=1)

In [7]:
# use matches that have a winner, ignore nc, draws 
df_train = df_train.drop(df_train[(df_train.winner != 1) & (df_train.winner != 2)].index)
df_train['winner'] = df_train['winner'].apply(lambda x: 1 if x == 1 else 0)


In [8]:
print(len(df_train.filter(regex="^fighter1",axis=1).columns))
print(len(df_train.filter(regex="^fighter2",axis=1).columns))
print(df_train["winner"].value_counts())
balance = int(6205 - (len(df_train)/2))
balance 

60
60
1    6205
0    2537
Name: winner, dtype: int64


1834

In [9]:
s1 = df_train.winner[df_train.winner.eq(1)].sample(balance).index

In [10]:
import re
newdf = df_train[df_train.index.isin(s1)]
newdf['winner'] = newdf['winner'].apply(lambda x: 0 if x == 1 else 1)
newdf = newdf.rename(columns=lambda x: re.sub('^fighter1','fighter3',x))
newdf = newdf.rename(columns=lambda x: re.sub('^fighter2','fighter1',x))
newdf = newdf.rename(columns=lambda x: re.sub('^fighter3','fighter2',x))
newdf = newdf.reindex(sorted(newdf.columns), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
df_train = df_train.drop(s1)
df_train = pd.concat([df_train, newdf], axis=0)

In [12]:
print(len(df_train.filter(regex="^fighter1",axis=1).columns))
print(len(df_train.filter(regex="^fighter2",axis=1).columns))
print(df_train["winner"].value_counts())

60
60
0    4371
1    4371
Name: winner, dtype: int64


In [13]:
# calculate age using birthyear of fighter
df_train['fighter1_birthyear'] = df_train['fighter1_birthyear'].replace( np.nan, 1992).astype('int64')
df_test['fighter1_birthyear'] = df_train['fighter1_birthyear'].replace( np.nan, 1992).astype('int64')
df_train['fighter2_birthyear'] = df_train['fighter2_birthyear'].replace( np.nan, 1992).astype('int64')
df_test['fighter2_birthyear'] = df_train['fighter2_birthyear'].replace( np.nan, 1992).astype('int64')

year_feats_birth = ['fighter1_birthyear','fighter2_birthyear']
year_feats_age = ['fighter1_age','fighter2_age']
this_year = 2022

# function for calculating age
def calc_age(birthyear):
    return this_year - birthyear
 
df_train[year_feats_age] = df_train[year_feats_birth].apply(calc_age)
df_test[year_feats_age] = df_test[year_feats_birth].apply(calc_age)

df_train = df_train.drop(columns = ["fighter1_birthyear","fighter2_birthyear"])
df_test = df_test.drop(columns = ["fighter1_birthyear","fighter2_birthyear"])

In [14]:
# calculate total fights for each fighter
fight1_feats = ['fighter1_wins','fighter1_losses','fighter1_draws','fighter1_ncs']
fight2_feats = ['fighter2_wins','fighter2_losses','fighter2_draws','fighter2_ncs']
fight_feats_new = ['fighter1_totalfights','fighter2_totalfights']

# function for total fights
def calc_fights(fights, fights2):
    return [sum(fights), sum(fights2)]

df_train[fight_feats_new] = df_train.apply(lambda x: calc_fights(x[fight1_feats],x[fight2_feats]), axis=1, result_type ='expand')
df_test[fight_feats_new] = df_test.apply(lambda x: calc_fights(x[fight1_feats],x[fight2_feats]), axis=1, result_type ='expand')

In [15]:
# convert total fight time from seconds to minutes
fight_feats_mins = ['fighter1_total_time_fought','fighter1_CTRL','fighter1_CTRLED','fighter2_total_time_fought','fighter2_CTRL','fighter2_CTRLED']
mins = 60

# function to convert from seconds to minutes
def convert_mins(time_fought):
    return time_fought / mins

df_train[fight_feats_mins] = df_train[fight_feats_mins].apply(convert_mins)
df_test[fight_feats_mins] = df_test[fight_feats_mins].apply(convert_mins)

# FEATURE ENGINEERING

In [16]:
#@title treatment for KD, REV, REVED, DOWNED

fight_stats_cols1 = ['fighter1_wsub_subatt', 'fighter1_rounds_fought_totalfights', 'fighter1_KD', 'fighter1_DOWNED', 'fighter1_SUBATT_CTRL', 'fighter1_REV_mins', 'fighter1_REVED_mins','fighter2_wsub_subatt', 'fighter2_rounds_fought_totalfights', 'fighter2_KD', 'fighter2_DOWNED', 'fighter2_SUBATT_CTRL', 'fighter2_REV_mins', 'fighter2_REVED_mins']

def add_cols1(row):
  try:
    fighter1_wsub_subatt = row.fighter1_wsub / row.fighter1_SUBATT
    fighter1_rounds_fought_totalfights = row.fighter1_rounds_fought / row.fighter1_totalfights
    fighter1_KD = row.fighter1_KD / row.fighter1_totalfights
    fighter1_DOWNED = row.fighter1_DOWNED / row.fighter1_totalfights
    fighter1_SUBATT_CTRL = row.fighter1_SUBATT / row.fighter1_CTRL
    fighter1_REV_mins = row.fighter1_REV / row.fighter1_CTRL
    fighter1_REVED_mins = row.fighter1_REVED / row.fighter1_CTRL

    fighter2_wsub_subatt = row.fighter2_wsub / row.fighter2_SUBATT
    fighter2_rounds_fought_totalfights = row.fighter2_rounds_fought / row.fighter2_totalfights
    fighter2_KD = row.fighter2_KD / row.fighter2_totalfights
    fighter2_DOWNED = row.fighter2_DOWNED / row.fighter2_totalfights
    fighter2_SUBATT_CTRL = row.fighter2_SUBATT / row.fighter2_CTRL
    fighter2_REV_mins = row.fighter2_REV / row.fighter2_CTRL
    fighter2_REVED_mins = row.fighter2_REVED / row.fighter2_CTRL
  except ZeroDivisionError:
    fighter1_wsub_subatt = 0 
    fighter1_rounds_fought_totalfights = 0 
    fighter1_KD = 0 
    fighter1_DOWNED = 0 
    fighter1_SUBATT_CTRL= 0 
    fighter1_REV_mins = 0
    fighter1_REVED_mins = 0

    fighter2_wsub_subatt = 0 
    fighter2_rounds_fought_totalfights = 0 
    fighter2_KD = 0
    fighter2_DOWNED = 0
    fighter2_SUBATT_CTRL = 0
    fighter2_REV_mins = 0
    fighter2_REVED_mins = 0
  return fighter1_wsub_subatt, fighter1_rounds_fought_totalfights, fighter1_KD, fighter1_DOWNED, fighter1_SUBATT_CTRL, fighter1_REV_mins, fighter1_REVED_mins,fighter2_wsub_subatt, fighter2_rounds_fought_totalfights, fighter2_KD, fighter2_DOWNED, fighter2_SUBATT_CTRL, fighter2_REV_mins, fighter2_REVED_mins

df_train[fight_stats_cols1] = df_train.apply(add_cols1, axis=1, result_type = 'expand')
df_test[fight_stats_cols1] = df_test.apply(add_cols1, axis=1, result_type = 'expand')

df_train = df_train.drop(columns = ['fighter1_SUBATT','fighter2_SUBATT','fighter1_REV','fighter2_REV','fighter1_REVED','fighter2_REVED'])
df_test = df_test.drop(columns = ['fighter1_SUBATT','fighter2_SUBATT','fighter1_REV','fighter2_REV','fighter1_REVED','fighter2_REVED'])


In [17]:
#@title treatement for dynamic stats with attempts
# calculate "attempt" features with per total fight mins, per attempt, and attempt per total fight mins

fight_stats_col2 = ['fighter1_SS_mins','fighter1_SS_att','fighter1_SSA_mins','fighter1_TD_mins','fighter1_TD_att','fighter1_TDA_mins','fighter1_HS_mins','fighter1_HS_att','fighter1_HSA_mins','fighter1_BS_mins','fighter1_BS_att','fighter1_BSA_mins','fighter1_LS_mins','fighter1_LS_att','fighter1_LSA_mins','fighter1_DS_mins','fighter1_DS_att','fighter1_DSA_mins','fighter1_CS_mins','fighter1_CS_att','fighter1_CSA_mins','fighter1_GS_mins','fighter1_GS_att','fighter1_GSA_mins','fighter1_SSD_mins','fighter1_SSR_mins','fighter1_TDD_mins','fighter1_TDD_TDR','fighter1_TDR_mins','fighter1_HSD_mins','fighter1_HSD_HSR','fighter1_HSR_mins','fighter1_BSD_mins','fighter1_BSD_BSR','fighter1_BSR_mins','fighter1_LSD_mins','fighter1_LSD_LSR','fighter1_LSR_mins','fighter1_DSD_mins','fighter1_DSD_DSR','fighter1_DSR_mins','fighter1_CSD_mins','fighter1_CSD_CSR','fighter1_CSR_mins','fighter1_GSD_mins','fighter1_CSD_GSR','fighter1_GSR_mins','fighter2_SS_mins','fighter2_SS_att','fighter2_SSA_mins','fighter2_TD_mins','fighter2_TD_att','fighter2_TDA_mins','fighter2_HS_mins','fighter2_HS_att','fighter2_HSA_mins','fighter2_BS_mins','fighter2_BS_att','fighter2_BSA_mins','fighter2_LS_mins','fighter2_LS_att','fighter2_LSA_mins','fighter2_DS_mins','fighter2_DS_att','fighter2_DSA_mins','fighter2_CS_mins','fighter2_CS_att','fighter2_CSA_mins','fighter2_GS_mins','fighter2_GS_att','fighter2_GSA_mins','fighter2_SSD_mins','fighter2_SSR_mins','fighter2_TDD_mins','fighter2_TDD_TDR','fighter2_TDR_mins','fighter2_HSD_mins','fighter2_HSD_HSR','fighter2_HSR_mins','fighter2_BSD_mins','fighter2_BSD_BSR','fighter2_BSR_mins','fighter2_LSD_mins','fighter2_LSD_LSR','fighter2_LSR_mins','fighter2_DSD_mins','fighter2_DSD_DSR','fighter2_DSR_mins','fighter2_CSD_mins','fighter2_CSD_CSR','fighter2_CSR_mins','fighter2_GSD_mins','fighter2_CSD_GSR','fighter2_GSR_mins']

def add_col2(row):
    try:
      fighter1_SS_mins = row.fighter1_SS / row.fighter1_total_time_fought
      fighter1_SS_att = row.fighter1_SS / row.fighter1_SSA
      fighter1_SSA_mins = row.fighter1_SSA / row.fighter1_total_time_fought
      fighter1_TD_mins = row.fighter1_TD / row.fighter1_total_time_fought
      fighter1_TD_att = row.fighter1_TD / row.fighter1_TDA
      fighter1_TDA_mins = row.fighter1_TD / row.fighter1_total_time_fought
      fighter1_HS_mins = row.fighter1_HS / row.fighter1_total_time_fought
      fighter1_HS_att = row.fighter1_HS / row.fighter1_HSA
      fighter1_HSA_mins = row.fighter1_HSA / row.fighter1_total_time_fought
      fighter1_BS_mins = row.fighter1_BS / row.fighter1_total_time_fought
      fighter1_BS_att= row.fighter1_BS / row.fighter1_BSA
      fighter1_BSA_mins = row.fighter1_BSA / row.fighter1_total_time_fought
      fighter1_LS_mins = row.fighter1_LS / row.fighter1_total_time_fought
      fighter1_LS_att = row.fighter1_LS / row.fighter1_LSA
      fighter1_LSA_mins = row.fighter1_LSA / row.fighter1_total_time_fought
      fighter1_DS_mins = row.fighter1_DS / row.fighter1_total_time_fought
      fighter1_DS_att = row.fighter1_DS / row.fighter1_DSA
      fighter1_DSA_mins = row.fighter2_DSA / row.fighter1_total_time_fought
      fighter1_CS_mins = row.fighter1_CS / row.fighter1_total_time_fought
      fighter1_CS_att = row.fighter1_CS / row.fighter1_CSA
      fighter1_CSA_mins = row.fighter1_CSA / row.fighter1_total_time_fought
      fighter1_GS_mins = row.fighter1_GS / row.fighter1_total_time_fought
      fighter1_GS_att = row.fighter1_GS / row.fighter1_GSA
      fighter1_GSA_mins = row.fighter1_GSA / row.fighter1_total_time_fought
      fighter1_SSD_mins = row.fighter1_SSD / row.fighter1_SSR
      fighter1_SSR_mins = row.fighter1_SSR / row.fighter1_total_time_fought
      fighter1_TDD_mins = row.fighter1_TDD / row.fighter1_total_time_fought
      fighter1_TDD_TDR = row.fighter1_TDD / row.fighter1_TDR
      fighter1_TDR_mins = row.fighter1_TDR / row.fighter1_total_time_fought
      fighter1_HSD_mins = row.fighter1_HSD / row.fighter1_total_time_fought
      fighter1_HSD_HSR = row.fighter1_HSD / row.fighter1_HSR
      fighter1_HSR_mins = row.fighter1_HSR / row.fighter1_total_time_fought
      fighter1_BSD_mins = row.fighter1_BSD / row.fighter1_total_time_fought
      fighter1_BSD_BSR = row.fighter1_BSD / row.fighter1_BSR
      fighter1_BSR_mins = row.fighter1_BSR / row.fighter1_total_time_fought
      fighter1_LSD_mins = row.fighter1_LSD / row.fighter1_total_time_fought
      fighter1_LSD_LSR = row.fighter1_LSD / row.fighter1_LSR
      fighter1_LSR_mins = row.fighter1_LSR / row.fighter1_total_time_fought
      fighter1_DSD_mins = row.fighter1_DSD / row.fighter1_total_time_fought
      fighter1_DSD_DSR = row.fighter1_DSD / row.fighter1_DSR
      fighter1_DSR_mins = row.fighter1_DSR / row.fighter1_total_time_fought
      fighter1_CSD_mins = row.fighter1_CSD / row.fighter1_total_time_fought
      fighter1_CSD_CSR = row.fighter1_CSD / row.fighter1_CSR
      fighter1_CSR_mins = row.fighter1_CSR / row.fighter1_total_time_fought
      fighter1_GSD_mins = row.fighter1_GSD / row.fighter1_total_time_fought
      fighter1_CSD_GSR = row.fighter1_GSD / row.fighter1_GSR
      fighter1_GSR_mins = row.fighter1_GSR / row.fighter1_total_time_fought

      fighter2_SS_mins = row.fighter2_SS / row.fighter2_total_time_fought
      fighter2_SS_att = row.fighter2_SS / row.fighter2_SSA
      fighter2_SSA_mins = row.fighter2_SSA / row.fighter2_total_time_fought
      fighter2_TD_mins = row.fighter2_TD / row.fighter2_total_time_fought
      fighter2_TD_att = row.fighter2_TD / row.fighter2_TDA
      fighter2_TDA_mins = row.fighter2_TD / row.fighter2_total_time_fought
      fighter2_HS_mins = row.fighter2_HS / row.fighter2_total_time_fought
      fighter2_HS_att = row.fighter2_HS / row.fighter2_HSA
      fighter2_HSA_mins = row.fighter2_HSA / row.fighter2_total_time_fought
      fighter2_BS_mins = row.fighter2_BS / row.fighter2_total_time_fought
      fighter2_BS_att= row.fighter2_BS / row.fighter2_BSA
      fighter2_BSA_mins = row.fighter2_BSA / row.fighter2_total_time_fought
      fighter2_LS_mins = row.fighter2_LS / row.fighter2_total_time_fought
      fighter2_LS_att = row.fighter2_LS / row.fighter2_LSA
      fighter2_LSA_mins = row.fighter2_LSA / row.fighter2_total_time_fought
      fighter2_DS_mins = row.fighter2_DS / row.fighter2_total_time_fought
      fighter2_DS_att = row.fighter2_DS / row.fighter2_DSA
      fighter2_DSA_mins = row.fighter2_DSA / row.fighter2_total_time_fought
      fighter2_CS_mins = row.fighter2_CS / row.fighter2_total_time_fought
      fighter2_CS_att = row.fighter2_CS / row.fighter2_CSA
      fighter2_CSA_mins = row.fighter2_CSA / row.fighter2_total_time_fought
      fighter2_GS_mins = row.fighter2_GS / row.fighter2_total_time_fought
      fighter2_GS_att = row.fighter2_GS / row.fighter2_GSA
      fighter2_GSA_mins = row.fighter2_GSA / row.fighter2_total_time_fought
      fighter2_SSD_mins = row.fighter2_SSD / row.fighter2_SSR
      fighter2_SSR_mins = row.fighter2_SSR / row.fighter2_total_time_fought
      fighter2_TDD_mins = row.fighter2_TDD / row.fighter2_total_time_fought
      fighter2_TDD_TDR = row.fighter2_TDD / row.fighter2_TDR
      fighter2_TDR_mins = row.fighter2_TDR / row.fighter2_total_time_fought
      fighter2_HSD_mins = row.fighter2_HSD / row.fighter2_total_time_fought
      fighter2_HSD_HSR = row.fighter2_HSD / row.fighter2_HSR
      fighter2_HSR_mins = row.fighter2_HSR / row.fighter2_total_time_fought
      fighter2_BSD_mins = row.fighter2_BSD / row.fighter2_total_time_fought
      fighter2_BSD_BSR = row.fighter2_BSD / row.fighter2_BSR
      fighter2_BSR_mins = row.fighter2_BSR / row.fighter2_total_time_fought
      fighter2_LSD_mins = row.fighter2_LSD / row.fighter2_total_time_fought
      fighter2_LSD_LSR = row.fighter2_LSD / row.fighter2_LSR
      fighter2_LSR_mins = row.fighter2_LSR / row.fighter2_total_time_fought
      fighter2_DSD_mins = row.fighter2_DSD / row.fighter2_total_time_fought
      fighter2_DSD_DSR = row.fighter2_DSD / row.fighter2_DSR
      fighter2_DSR_mins = row.fighter2_DSR / row.fighter2_total_time_fought
      fighter2_CSD_mins = row.fighter2_CSD / row.fighter2_total_time_fought
      fighter2_CSD_CSR = row.fighter2_CSD / row.fighter2_CSR
      fighter2_CSR_mins = row.fighter2_CSR / row.fighter2_total_time_fought
      fighter2_GSD_mins = row.fighter2_GSD / row.fighter2_total_time_fought
      fighter2_CSD_GSR = row.fighter2_GSD / row.fighter2_GSR
      fighter2_GSR_mins = row.fighter2_GSR / row.fighter2_total_time_fought
    except ZeroDivisionError:
      fighter1_SS_mins = 0
      fighter1_SS_att = 0
      fighter1_SSA_mins = 0
      fighter1_TD_mins = 0
      fighter1_TD_att = 0
      fighter1_TDA_mins= 0
      fighter1_HS_mins= 0
      fighter1_HS_att= 0
      fighter1_HSA_mins= 0
      fighter1_BS_mins= 0
      fighter1_BS_att= 0
      fighter1_BSA_mins= 0
      fighter1_LS_mins= 0
      fighter1_LS_att= 0
      fighter1_LSA_mins= 0
      fighter1_DS_mins= 0
      fighter1_DS_att= 0
      fighter2_DSA_mins= 0
      fighter1_CS_mins= 0
      fighter1_CS_att= 0
      fighter1_CSA_mins= 0
      fighter1_GS_mins= 0
      fighter1_GS_att= 0
      fighter1_GSA_mins= 0
      fighter1_SSD_mins= 0
      fighter1_SSR_mins= 0
      fighter1_TDD_mins= 0
      fighter1_TDD_TDR= 0
      fighter1_TDR_mins= 0
      fighter1_HSD_mins= 0
      fighter1_HSD_HSR= 0
      fighter1_HSR_mins= 0
      fighter1_BSD_mins= 0
      fighter1_BSD_BSR= 0
      fighter1_BSR_mins= 0
      fighter1_LSD_mins= 0
      fighter1_LSD_LSR= 0
      fighter1_LSR_mins= 0
      fighter1_DSD_mins= 0
      fighter1_DSD_DSR= 0
      fighter1_DSR_mins= 0
      fighter1_CSD_mins= 0
      fighter1_CSD_CSR= 0
      fighter1_CSR_mins= 0
      fighter1_GSD_mins= 0
      fighter1_CSD_GSR= 0
      fighter1_GSR_mins= 0
      fighter2_SS_mins= 0
      fighter2_SS_att= 0
      fighter2_SSA_mins= 0
      fighter2_TD_mins= 0
      fighter2_TD_att= 0
      fighter2_TDA_mins= 0
      fighter2_HS_mins= 0
      fighter2_HS_att= 0
      fighter2_HSA_mins= 0
      fighter2_BS_mins= 0
      fighter2_BS_att= 0
      fighter2_BSA_mins= 0
      fighter2_LS_mins= 0
      fighter2_LS_att= 0
      fighter2_LSA_mins= 0
      fighter2_DS_mins= 0
      fighter2_DS_att= 0
      fighter2_DSA_mins= 0
      fighter2_CS_mins= 0
      fighter2_CS_att= 0
      fighter2_CSA_mins= 0
      fighter2_GS_mins= 0
      fighter2_GS_att= 0
      fighter2_GSA_mins= 0
      fighter2_SSD_mins= 0
      fighter2_SSR_mins= 0
      fighter2_TDD_mins= 0
      fighter2_TDD_TDR= 0
      fighter2_TDR_mins= 0
      fighter2_HSD_mins= 0
      fighter2_HSD_HSR= 0
      fighter2_HSR_mins= 0
      fighter2_BSD_mins= 0
      fighter2_BSD_BSR= 0
      fighter2_BSR_mins= 0
      fighter2_LSD_mins= 0
      fighter2_LSD_LSR= 0
      fighter2_LSR_mins= 0
      fighter2_DSD_mins= 0
      fighter2_DSD_DSR= 0
      fighter2_DSR_mins= 0
      fighter2_CSD_mins= 0
      fighter2_CSD_CSR=0
      fighter2_CSR_mins=0
      fighter2_GSD_mins=0
      fighter2_CSD_GSR= 0
      fighter2_GSR_mins=0
    return fighter1_SS_mins,fighter1_SS_att,fighter1_SSA_mins,fighter1_TD_mins,fighter1_TD_att,fighter1_TDA_mins,fighter1_HS_mins,fighter1_HS_att,fighter1_HSA_mins,fighter1_BS_mins,fighter1_BS_att,fighter1_BSA_mins,fighter1_LS_mins,fighter1_LS_att,fighter1_LSA_mins,fighter1_DS_mins,fighter1_DS_att,fighter2_DSA_mins,fighter1_CS_mins,fighter1_CS_att,fighter1_CSA_mins,fighter1_GS_mins,fighter1_GS_att,fighter1_GSA_mins,fighter1_SSD_mins,fighter1_SSR_mins,fighter1_TDD_mins,fighter1_TDD_TDR,fighter1_TDR_mins,fighter1_HSD_mins,fighter1_HSD_HSR,fighter1_HSR_mins,fighter1_BSD_mins,fighter1_BSD_BSR,fighter1_BSR_mins,fighter1_LSD_mins,fighter1_LSD_LSR,fighter1_LSR_mins,fighter1_DSD_mins,fighter1_DSD_DSR,fighter1_DSR_mins,fighter1_CSD_mins,fighter1_CSD_CSR,fighter1_CSR_mins,fighter1_GSD_mins,fighter1_CSD_GSR,fighter1_GSR_mins,fighter2_SS_mins,fighter2_SS_att,fighter2_SSA_mins,fighter2_TD_mins,fighter2_TD_att,fighter2_TDA_mins,fighter2_HS_mins,fighter2_HS_att,fighter2_HSA_mins,fighter2_BS_mins,fighter2_BS_att,fighter2_BSA_mins,fighter2_LS_mins,fighter2_LS_att,fighter2_LSA_mins,fighter2_DS_mins,fighter2_DS_att,fighter2_DSA_mins,fighter2_CS_mins,fighter2_CS_att,fighter2_CSA_mins,fighter2_GS_mins,fighter2_GS_att,fighter2_GSA_mins,fighter2_SSD_mins,fighter2_SSR_mins,fighter2_TDD_mins,fighter2_TDD_TDR,fighter2_TDR_mins,fighter2_HSD_mins,fighter2_HSD_HSR,fighter2_HSR_mins,fighter2_BSD_mins,fighter2_BSD_BSR,fighter2_BSR_mins,fighter2_LSD_mins,fighter2_LSD_LSR,fighter2_LSR_mins,fighter2_DSD_mins,fighter2_DSD_DSR,fighter2_DSR_mins,fighter2_CSD_mins,fighter2_CSD_CSR,fighter2_CSR_mins,fighter2_GSD_mins,fighter2_CSD_GSR,fighter2_GSR_mins

df_train[fight_stats_col2] = df_train.apply(add_col2, axis=1, result_type = 'expand')
df_test[fight_stats_col2] = df_test.apply(add_col2, axis=1, result_type = 'expand')

df_train = df_train.drop(columns = ['fighter1_SS','fighter1_SSA','fighter2_SS','fighter2_SSA','fighter1_TD','fighter1_TDA','fighter2_TD','fighter2_TDA','fighter1_HS','fighter1_HSA','fighter2_HS','fighter2_HSA','fighter1_BS','fighter1_BSA','fighter2_BS','fighter2_BSA','fighter1_LS','fighter1_LSA','fighter2_LS','fighter2_LSA','fighter1_DS','fighter1_DSA','fighter2_DS','fighter2_DSA','fighter1_CS','fighter1_CSA','fighter2_CS','fighter2_CSA','fighter1_GS','fighter1_GSA','fighter2_GS','fighter2_GSA','fighter1_SSD','fighter1_SSR','fighter2_SSD','fighter2_SSR','fighter1_TDD','fighter1_TDR','fighter2_TDD','fighter2_TDR','fighter1_HSD','fighter1_HSR','fighter2_HSD','fighter2_HSR','fighter1_BSD','fighter1_BSR','fighter2_BSD','fighter2_BSR','fighter1_LSD','fighter1_LSR','fighter2_LSD','fighter2_LSR','fighter1_DSD','fighter1_DSR','fighter2_DSD','fighter2_DSR','fighter1_CSD','fighter1_CSR','fighter2_CSD','fighter2_CSR','fighter1_GSD','fighter1_GSR','fighter2_GSD','fighter2_GSR'])
df_test = df_test.drop(columns = ['fighter1_SS','fighter1_SSA','fighter2_SS','fighter2_SSA','fighter1_TD','fighter1_TDA','fighter2_TD','fighter2_TDA','fighter1_HS','fighter1_HSA','fighter2_HS','fighter2_HSA','fighter1_BS','fighter1_BSA','fighter2_BS','fighter2_BSA','fighter1_LS','fighter1_LSA','fighter2_LS','fighter2_LSA','fighter1_DS','fighter1_DSA','fighter2_DS','fighter2_DSA','fighter1_CS','fighter1_CSA','fighter2_CS','fighter2_CSA','fighter1_GS','fighter1_GSA','fighter2_GS','fighter2_GSA','fighter1_SSD','fighter1_SSR','fighter2_SSD','fighter2_SSR','fighter1_TDD','fighter1_TDR','fighter2_TDD','fighter2_TDR','fighter1_HSD','fighter1_HSR','fighter2_HSD','fighter2_HSR','fighter1_BSD','fighter1_BSR','fighter2_BSD','fighter2_BSR','fighter1_LSD','fighter1_LSR','fighter2_LSD','fighter2_LSR','fighter1_DSD','fighter1_DSR','fighter2_DSD','fighter2_DSR','fighter1_CSD','fighter1_CSR','fighter2_CSD','fighter2_CSR','fighter1_GSD','fighter1_GSR','fighter2_GSD','fighter2_GSR'])


In [18]:

fight_stats_col3 = ['fighter1_win_pct','fighter1_wko','fighter1_wsub','fighter1_wdec','fighter1_wdq','fighter1_lko','fighter1_lsub','fighter1_ldec','fighter1_ldq','fighter2_win_pct','fighter2_wko','fighter2_wsub','fighter2_wdec','fighter2_wdq','fighter2_lko','fighter2_lsub','fighter2_ldec','fighter2_ldq']

def add_col3(row):
    try:
      fighter1_win_pct = row.fighter1_wins / row.fighter1_totalfights
      fighter1_wko = row.fighter1_wko / row.fighter1_wins
      fighter1_wsub = row.fighter1_wsub / row.fighter1_wins
      fighter1_wdec = row.fighter1_wdec / row.fighter1_wins
      fighter1_wdq = row.fighter1_wdq / row.fighter1_wins
      fighter1_lko = row.fighter1_lko / row.fighter1_losses
      fighter1_lsub = row.fighter1_lsub / row.fighter1_losses
      fighter1_ldec = row.fighter1_ldec/ row.fighter1_losses
      fighter1_ldq = row.fighter1_ldq/ row.fighter1_losses

      fighter2_win_pct = row.fighter2_wins / row.fighter2_totalfights
      fighter2_wko = row.fighter2_wko / row.fighter2_wins
      fighter2_wsub = row.fighter2_wsub / row.fighter2_wins
      fighter2_wdec = row.fighter2_wdec / row.fighter2_wins
      fighter2_wdq = row.fighter2_wdq / row.fighter2_wins
      fighter2_lko = row.fighter2_lko / row.fighter2_losses
      fighter2_lsub = row.fighter2_lsub / row.fighter2_losses
      fighter2_ldec = row.fighter2_ldec/ row.fighter2_losses
      fighter2_ldq = row.fighter2_ldq/ row.fighter2_losses

    except ZeroDivisionError:
      fighter1_win_pct = 0
      fighter1_wko = 0
      fighter1_wsub=0
      fighter1_wdec=0
      fighter1_wdq=0
      fighter1_lko=0
      fighter1_lsub=0
      fighter1_ldec=0
      fighter1_ldq=0

      fighter2_win_pct = 0
      fighter2_wko = 0
      fighter2_wsub=0
      fighter2_wdec=0
      fighter2_wdq=0
      fighter2_lko=0
      fighter2_lsub=0
      fighter2_ldec=0
      fighter2_ldq=0

    return fighter1_win_pct,fighter1_wko,fighter1_wsub,fighter1_wdec,fighter1_wdq,fighter1_lko,fighter1_lsub,fighter1_ldec,fighter1_ldq,fighter2_win_pct,fighter2_wko,fighter2_wsub,fighter2_wdec,fighter2_wdq,fighter2_lko,fighter2_lsub,fighter2_ldec,fighter2_ldq

df_train[fight_stats_col3] = df_train.apply(add_col3, axis=1, result_type = 'expand')
df_test[fight_stats_col3] = df_test.apply(add_col3, axis=1, result_type = 'expand')


In [19]:
df_train = df_train.reindex(sorted(df_train.columns), axis=1)
df_test = df_test.reindex(sorted(df_test.columns), axis=1)

In [20]:
from google.colab import files
df_train.to_csv('UFC_pred.csv') 
files.download('UFC_pred.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
# split the dataset
from sklearn.model_selection import train_test_split

df_train = df_train.drop(columns = ['_id','fighter1_id','fighter2_id','fighter1_name','fighter2_name','method','weight_class', 'year','end_round'])
df_test = df_test.drop(columns = ['_id','fighter1_id','fighter2_id','fighter1_name','fighter2_name','method','weight_class', 'year','end_round'])

X, y = df_train.drop("winner", axis=1), df_train["winner"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder


num_feats = list(X_train.select_dtypes('number').columns.values)
stance_feats = ['fighter1_stance','fighter2_stance']
null_feats = ['fighter1_height','fighter1_weight','fighter1_reach','fighter2_height','fighter2_weight','fighter2_reach']


stance_imputer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encode',OrdinalEncoder())])

iter_imputer = Pipeline(steps=[
    ('iter_imputer',IterativeImputer())])

scaler = Pipeline(steps=[('scaler',MinMaxScaler())])

#transform columns
categorical_col = ['fighter1_stance','fighter2_stance']
preprocessor = ColumnTransformer(transformers = [('stance_impute',stance_imputer, stance_feats),
                                                 ('iter_imputer',iter_imputer, null_feats),
                                                 ('scaler', scaler,num_feats),
                                                #  ],remainder='passthrough')
                                                ])

preprocessor.fit(X_train)

X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)
df_test_processed = preprocessor.transform(df_test)

X_train1 = pd.DataFrame(preprocessor.transform(X_train), columns = X_train.columns)
X_test1 = pd.DataFrame(preprocessor.transform(X_test),columns = X_test.columns)
# df_test1 = pd.DataFrame(preprocessor.transform(df_test),columns = df_test.columns)

ValueError: ignored

In [22]:
from google.colab import files
df_test1.to_csv('UFC_pred.csv') 
files.download('UFC_pred.csv')

# MODELING WITH RNN

In [23]:
# conts = X.drop(cats, axis=1).columns.values.tolist()

In [24]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6119, 154) (2623, 154) (6119,) (2623,)


In [25]:
print(y_train.mean(), y_test.mean())

0.49812060794247426 0.5043842927945101


In [26]:
pd.isna(X_train).sum()[pd.isna(X_train).sum() >0]

fighter1_height      15
fighter1_reach     1011
fighter1_stance      88
fighter1_weight       5
fighter2_height      33
fighter2_reach     1211
fighter2_stance     108
fighter2_weight      12
dtype: int64

In [27]:
# evaluate iterative imputation and random forest for the horse colic dataset
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [28]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, log_loss

In [29]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, log_loss
# clf = DecisionTreeClassifier(random_state=0, criterion="entropy", max_depth=5)
# class_names = [str(x) for x in clf.classes_]

nn_clf = MLPClassifier(solver='lbfgs', activation='relu', alpha=1e-3, max_iter=1500,
                       hidden_layer_sizes=(10, 10), random_state=1, verbose=True)
nn_clf.fit(X_train_processed, y_train)

y_pred_nn = nn_clf.predict(X_test_processed)
outputs = nn_clf.predict(df_test_processed)
print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred_nn)))

Accuracy = 0.83


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [31]:
outputs = pd.Series(outputs) 
pd.concat([df_test_pred['fighter1_name'], df_test_pred['fighter2_name'], outputs], axis=1)

,fighter1_name,fighter2_name,0
0,Charles Oliveira,Islam Makhachev,0
1,Aljamain Sterling,TJ Dillashaw,0
2,Petr Yan,Sean O'Malley,0
3,Beneil Dariush,Mateusz Gamrot,0
4,Katlyn Chookagian,Manon Fiorot,0
5,Belal Muhammad,Sean Brady,0
6,Makhmud Muradov,Caio Borralho,0
7,Volkan Oezdemir,Nikita Krylov,0
8,Abubakar Nurmagomedov,Gadzhi Omargadzhiev,0
9,Armen Petrosyan,AJ Dobson,1


In [ ]:
# from sklearn.tree import XGBClassifier as xgboost
import xgboost 

xgclassifier = xgboost.XGBClassifier(use_label_encoder=False,eval_metric="auc",seed=0)


# Cross validate using auc as our metric and print out average of all folds
cv_scores = cross_val_score(xgclassifier, X_train_processed, y_train, cv=5, scoring='roc_auc')
# print("\nxgb Model, K={:3d} CV, ROC AUC: {:0.3f} +/- {:0.2f}".format(15, np.mean(cv_scores)))
print(np.mean(cv_scores))

# Now fit xgboost H1N1 classifier and predict probabiliities
xgclassifier.fit(X_train_processed, y_train)
y_pred_h1n1 = xgclassifier.predict_proba(X_test_processed)
output = xgclassifier.predict_proba(df_test_processed)


In [39]:
# LSTM and CNN for sequence classification in the IMDB dataset
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
tf.random.set_seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
# X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
# X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
# model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Embedding(5000,embedding_vecor_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
print(model.summary())
model.fit(X_train_processed, y_train, epochs=5, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test_processed, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          160000    
                                                                 
 conv1d_1 (Conv1D)           (None, None, 32)          3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, None, 32)         0         
 1D)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 100)               53200     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
________________________________________________

In [30]:
import numpy as np
import tensorflow as tf
# import tensorflow._api.v1.keras.backend as K
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.metrics import r2_score as r2

# from utils import r2, random_data_shuffle, get_train_test_data
    
epochs = 1050           
hidden1 = 350
dropout = 0.45
l2_reg = l2(0.005)

model = tf.keras.models.Sequential()
model.add(Dropout(dropout))
model.add(Dense(hidden1, input_dim=X_train.shape[1], activation='relu',
                    kernel_initializer='normal', kernel_regularizer=l2_reg))

model.add(Dense(y_train.shape[0], activation='linear'))  
# model.add(Dense(y_train.shape, activation='linear')) 

model.compile(
        loss='mse',
        optimizer=tf.keras.optimizers.Adam(0.0001),
        metrics=[r2]
    )

history = model.fit(X_train, y_train,
                        epochs=epochs, batch_size=32,
                        # validation_split=0.1,
                        # shuffle=True
                        )

scores = model.evaluate(X_test, y_test)
print(scores)

Epoch 1/1050


NotImplementedError: ignored

In [ ]:
y_train.shape[0]

6119

In [ ]:
class UfcDataset(Dataset):
    def __init__(self, df, cats, conts, targets):
        self.X_cats = df[cats].astype(np.int64).values
        self.X_conts = df[conts].astype(np.float32).values
        self.y = targets.astype(np.float32).values.reshape(-1, 1)
        
        
    def __len__(self):
        return self.y.shape[0]
    
    def __getitem__(self, i):
        return [self.X_cats[i], self.X_conts[i], self.y[i]]

In [ ]:

train_dataset = UfcDataset(X_train, conts, conts, y_train)
test_dataset = UfcDataset(X_test, conts, conts, y_test)

NameError: ignored

In [ ]:
# prediction

y = np.array([1,1,1,1,1,1,1,1,1,1,1])
y = pd.Series(y)

test_dataset = UfcDataset(X, cats, conts, y)

In [ ]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:

# Data loaders
bz=32
# train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=bz, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=bz, shuffle=False)

In [ ]:
class UfcNet(nn.Module):
    def __init__(self, emb_dims, num_conts, fc_layer_sizes, emb_drop, ps):
        super(UfcNet, self).__init__()
        
        # embedding layers for categorical features
        self.emb_layers = nn.ModuleList([nn.Embedding(x, y) for x, y in emb_dims])
        self.num_embs = sum([y for _, y in emb_dims])
        self.num_conts = num_conts
        
        # fully connected layers
        fc_layer_sizes = [self.num_embs + self.num_conts] + fc_layer_sizes
        self.fc_layers = nn.ModuleList([nn.Linear(fc_layer_sizes[i],fc_layer_sizes[i+1])
                                        for i in range(len(fc_layer_sizes)-1)])
        
        # out layer
        self.out = nn.Linear(fc_layer_sizes[-1], 1)
        
        # batch norm layers
        self.first_bn = nn.BatchNorm1d(self.num_conts)
        self.bn_layers = nn.ModuleList([nn.BatchNorm1d(sz)
                                        for sz in fc_layer_sizes[1:]])
        # dropout layers
        self.emb_drop = nn.Dropout(emb_drop)
        self.dropout_layers = nn.ModuleList([nn.Dropout(p) for p in ps])
        
    def forward(self, x_cats, x_conts):
        x = [e(x_cats[:, i]) for i, e in enumerate(self.emb_layers)]
        x = torch.cat(x, 1)
        x = self.emb_drop(x)
        
        x_c = self.first_bn(x_conts)
        x = torch.cat([x, x_c], 1)
        
        for fc, bn, d in zip(self.fc_layers, self.bn_layers, self.dropout_layers):
            x = F.relu(fc(x))
            x = bn(x)
            x = d(x)
        
        x = self.out(x)
        return torch.sigmoid(x)

In [ ]:
emb_dims = [(len(data[cat].unique()), min(50, len(data[cat].unique())//2)) for cat in cats]
emb_dims

[(3, 1), (3, 1)]

In [ ]:
num_conts = len(conts)
fc_layer_sizes = [256, 64, 16]
emb_drop = 0.5
ps = [0.5] * 3

In [ ]:
ufc_model = UfcNet(emb_dims, num_conts, fc_layer_sizes, emb_drop, ps).to(device)

In [ ]:

criterion = nn.BCELoss()
learning_rate = 1e-2
optimizer = torch.optim.Adam(ufc_model.parameters(), lr=learning_rate, weight_decay=1e-4)  

In [ ]:
num_epochs = 4
total_step = len(test_dl)
for epoch in range(num_epochs):
    for i, (x_cats, x_conts, y) in enumerate(test_dl):
        x_cats, x_conts, y = x_cats.to(device), x_conts.to(device), y.to(device)
        
        # forward
        outputs = ufc_model(x_cats, x_conts)
        loss = criterion(outputs, y)
        
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for x_cats, x_conts, y in test_dl:
        x_cats, x_conts, y = x_cats.to(device), x_conts.to(device), y.to(device)
        outputs = ufc_model(x_cats, x_conts)
        preds = (outputs>0.5).type(torch.FloatTensor)
        total += y.size(0)
        correct += (preds == y).sum().item()

print ("Accuracy: {:.2f}%".format(100*correct/total))

Accuracy: 81.82%


In [44]:
outputs = pd.Series(outputs) 

In [45]:
pd.concat([df_test_pred['fighter1_name'], df_test_pred['fighter2_name'], outputs], axis=1)

,fighter1_name,fighter2_name,0
0,Alexa Grasso,Viviane Araujo,1
1,Cub Swanson,Jonathan Martinez,0
2,Askar Askarov,Brandon Royval,1
3,Jordan Wright,Dusko Todorovic,0
4,Misha Cirkunov,Alonzo Menifield,0
5,Mana Martinez,Brandon Davis,0
6,Raphael Assuncao,Victor Henry,1
7,Nick Maximov,Jacob Malkoun,1
8,Piera Rodriguez,Sam Hughes,1
9,Tatsuro Taira,CJ Vergara,1


In [ ]:
data1

In [ ]:
X